In [1]:
## what effect does time have on location, district, race, age 

In [2]:
import pandas as pd
from lib import standardize_item_numbers

def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["footage_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [3]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_8560\2401213272.py:5: DtypeWarning: Columns (14,20,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [4]:
df.item_number.unique()

array(['d2077118', 'e1274818', 'l2194418', ..., 'j2225922', 'a1349122',
       'a3158222'], dtype=object)

In [5]:
df = df.drop_duplicates(subset=["offender_race", "offender_gender", "offender_age", "item_number"])

In [6]:
datetimes = (df.occurred_date_time.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1").str.extract(r"(\w{2}):(\w{2}):?(\w{2})?"))
df.loc[:, "occurred_datetime"] = datetimes[0]
df.occurred_datetime.value_counts()

C:\Users\PC\AppData\Local\Temp\ipykernel_8560\43808543.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  datetimes = (df.occurred_date_time.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1").str.extract(r"(\w{2}):(\w{2}):?(\w{2})?"))


23    487
21    454
20    448
19    431
01    428
17    423
18    412
22    403
16    381
00    370
15    370
02    369
03    337
11    336
14    330
12    303
10    286
04    264
13    259
05    159
09    159
08    151
06    138
07    121
Name: occurred_datetime, dtype: int64

In [7]:
df.location.count().sum()

8488

In [8]:
df.shape

(8488, 29)

In [9]:
requested = df[df.footage_requested.isin(["1"])]

In [10]:
requested.year.value_counts()

2021    2095
2020    1989
2022    1770
2019    1605
2018    1029
Name: year, dtype: int64

In [11]:
requested.groupby("year").disposition.value_counts()

year  disposition
2018  OPEN            612
      CLOSED          396
      UNFOUNDED        21
2019  OPEN            938
      CLOSED          637
      UNFOUNDED        30
2020  OPEN           1423
      CLOSED          530
      UNFOUNDED        36
2021  OPEN           1545
      CLOSED          499
      UNFOUNDED        51
2022  OPEN           1276
      CLOSED          472
      UNFOUNDED        22
Name: disposition, dtype: int64

In [12]:
requested.groupby("year").disposition.value_counts(normalize=True)

year  disposition
2018  OPEN           0.594752
      CLOSED         0.384840
      UNFOUNDED      0.020408
2019  OPEN           0.584424
      CLOSED         0.396885
      UNFOUNDED      0.018692
2020  OPEN           0.715435
      CLOSED         0.266466
      UNFOUNDED      0.018100
2021  OPEN           0.737470
      CLOSED         0.238186
      UNFOUNDED      0.024344
2022  OPEN           0.720904
      CLOSED         0.266667
      UNFOUNDED      0.012429
Name: disposition, dtype: float64

In [13]:
# r_ct = requested.groupby("year").disposition.value_counts().to_excel("../data/real_time_crime_center/requested_by_year_ct.xlsx")

In [14]:
# r_pct = requested.groupby("year").disposition.value_counts(normalize=True).to_excel("../data/real_time_crime_center/requested_by_year_pct.xlsx")

In [15]:
not_requested = df[~((df.footage_requested == "1.0"))]

In [16]:
not_requested.groupby("year").disposition.value_counts()

year  disposition
2018  OPEN            612
      CLOSED          396
      UNFOUNDED        21
2019  OPEN            938
      CLOSED          637
      UNFOUNDED        30
2020  OPEN           1423
      CLOSED          530
      UNFOUNDED        36
2021  OPEN           1545
      CLOSED          499
      UNFOUNDED        51
2022  OPEN           1276
      CLOSED          472
      UNFOUNDED        22
Name: disposition, dtype: int64

In [17]:
not_requested.groupby("year").disposition.value_counts(normalize=True)

year  disposition
2018  OPEN           0.594752
      CLOSED         0.384840
      UNFOUNDED      0.020408
2019  OPEN           0.584424
      CLOSED         0.396885
      UNFOUNDED      0.018692
2020  OPEN           0.715435
      CLOSED         0.266466
      UNFOUNDED      0.018100
2021  OPEN           0.737470
      CLOSED         0.238186
      UNFOUNDED      0.024344
2022  OPEN           0.720904
      CLOSED         0.266667
      UNFOUNDED      0.012429
Name: disposition, dtype: float64

In [18]:
r_ct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts())
r_ct  = r_ct.rename(columns={r_ct.columns[0]: "requested_ct"})
r_ct = r_ct.reset_index()

r_pct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts(normalize=True))
r_pct  = r_pct.rename(columns={r_pct.columns[0]: "requested_pct"})
r_pct = r_pct.reset_index()

nr_ct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts())
nr_ct  = nr_ct.rename(columns={nr_ct.columns[0]: "not_requested_ct"})
nr_ct = nr_ct.reset_index()

nr_pct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts(normalize=True))
nr_pct  = nr_pct.rename(columns={nr_pct.columns[0]: "not_requested_pct"})
nr_pct = nr_pct.reset_index()


In [19]:
dfs = pd.concat([nr_pct, nr_ct, r_pct, r_ct])

In [20]:
dfs

,year,disposition,not_requested_pct,not_requested_ct,requested_pct,requested_ct
0,2018,OPEN,0.594752,NaN,NaN,NaN
1,2018,CLOSED,0.384840,NaN,NaN,NaN
2,2018,UNFOUNDED,0.020408,NaN,NaN,NaN
3,2019,OPEN,0.584424,NaN,NaN,NaN
4,2019,CLOSED,0.396885,NaN,NaN,NaN
5,2019,UNFOUNDED,0.018692,NaN,NaN,NaN
6,2020,OPEN,0.715435,NaN,NaN,NaN
7,2020,CLOSED,0.266466,NaN,NaN,NaN
8,2020,UNFOUNDED,0.018100,NaN,NaN,NaN
9,2021,OPEN,0.737470,NaN,NaN,NaN


In [21]:
# df.to_excel("../data/real_time_crime_center/requests_per_year_2018_present.xlsx")